# 用subprocess模块来管理子进程

In [1]:
import logging
from sys import stdout as STDOUT

## 示例1：用Popen构造器来启动进程，然后用communicate方法读取子进程的输出信息，并等待其终止。

In [2]:
import subprocess
proc = subprocess.Popen(
    ['echo', 'Hello from the child!'],
    shell=True,
    stdout=subprocess.PIPE)
out, err = proc.communicate()
print(out.decode('utf-8'))

"Hello from the child!"



## 示例2：一边定期查询子进程的状态，一边处理其他事务

In [3]:
from time import sleep, time
proc = subprocess.Popen(['timeout', '1000'], shell=True)
while proc.poll() is None:
    print('Working...')
    # Some time consuming work here
    sleep(0.2)

print('Exit status', proc.poll())

Working...
Working...
Exit status 1


## 示例3：把子进程从父进程中剥离，父进程可以随意运行很多条平行的子进程。

In [4]:
def run_sleep(period):
    proc = subprocess.Popen(['timeout', str(period)], shell=True)
    return proc

In [5]:
start = time()
procs = []
for _ in range(10):
    proc = run_sleep(0.1)
    procs.append(proc)

In [6]:
for proc in procs:
    proc.communicate()
end = time()
print('Finished in %.3f seconds' % (end - start))

Finished in 0.632 seconds


## 示例4：要用命令行式的openssl工具加密一些数据。

In [7]:
import os

In [8]:
def run_openssl(data):
    env = os.environ.copy()
    env['password'] = str(b'\xe24U\n\xd0Ql3S\x11')
    proc = subprocess.Popen(
        ['openssl', 'enc', '-des3', '-pass', 'env:password'],
        env=env,
        shell=True,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE)
    proc.stdin.write(data)
    proc.stdin.flush()  # Ensure the child gets input
    return proc

In [9]:
procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    procs.append(proc)

In [10]:
for proc in procs:
    out, err = proc.communicate()
    print(out[-10:])

b'\x92\x80|54XN\xdc\xfc\xde'
b'\x9b\xa0$\x9d6\xc7.\x82\x9b\xa5'
b'\xbf\x91\xd8\xab\xd4\xa4\x1cC\xf2\xb9'


## 示例5：启动一个子进程，该进程会用命令行式的md5工具来处理输入流中的数据。

In [11]:
def run_md5(input_stdin):
    proc = subprocess.Popen(
        ['certutil', 'MD5'],
        shell=True,
        stdin=input_stdin,
        stdout=subprocess.PIPE)
    return proc

In [12]:
input_procs = []
hash_procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    input_procs.append(proc)
    hash_proc = run_md5(proc.stdout)
    hash_procs.append(hash_proc)

In [13]:
for proc in input_procs:
    proc.communicate()
for proc in hash_procs:
    out, err = proc.communicate()
    print(out.strip())

b'CertUtil: -dump \xca\xa7\xb0\xdc: 0x80070002 (WIN32: 2)\r\nCertUtil: \xcf\xb5\xcd\xb3\xd5\xd2\xb2\xbb\xb5\xbd\xd6\xb8\xb6\xa8\xb5\xc4\xce\xc4\xbc\xfe\xa1\xa3'
b'CertUtil: -dump \xca\xa7\xb0\xdc: 0x80070002 (WIN32: 2)\r\nCertUtil: \xcf\xb5\xcd\xb3\xd5\xd2\xb2\xbb\xb5\xbd\xd6\xb8\xb6\xa8\xb5\xc4\xce\xc4\xbc\xfe\xa1\xa3'
b'CertUtil: -dump \xca\xa7\xb0\xdc: 0x80070002 (WIN32: 2)\r\nCertUtil: \xcf\xb5\xcd\xb3\xd5\xd2\xb2\xbb\xb5\xbd\xd6\xb8\xb6\xa8\xb5\xc4\xce\xc4\xbc\xfe\xa1\xa3'


In [14]:
proc = run_sleep(10)
try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()

print('Exit status', proc.poll())

Exit status 1
